In [1]:
"""
@author: brianwsyang
@date: Sat Jul 27, 2019
"""

import pandas as pd
import numpy as np
import re

In [2]:
num_cou = 3007

In [3]:
# creates pd named couid
# contains data for county id, county, county-st

In [4]:
coucsv = pd.read_csv('counties.csv')
statecsv = pd.read_csv('50_us_states_all_data.csv')

cou_st = pd.merge(statecsv,
                    coucsv,
                    on='State')

cou_st = cou_st.rename({'Unnamed: 0': 'COU-ST'}, axis='columns')

# print(any(cou_st['COU-ST'].duplicated()))

cou = cou_st['County'].tolist()
state = cou_st['Abbr'].tolist()
ind = cou_st['COU-ST'].tolist()


for i in np.arange(num_cou):
    c_st = cou[i] + ', ' + state[i]
    cou_st = cou_st.replace({'COU-ST': {ind[i]: c_st}})

In [5]:
couid = cou_st[['CountyId', 'COU-ST']]

In [6]:
# create pd named crm using csv
# clean crm

In [7]:
crm = pd.read_csv('crime_data_w_population_and_crime_rate.csv')

In [8]:
crm = crm.drop(['index', 'EDITION', 'PART'], axis=1)
cou_crm = crm[crm['county_name'].str.contains('County')]

In [9]:
cou_head = cou_crm.columns.tolist()

da_cou = cou_crm.mean(axis=0).tolist()
test = cou_crm.max(axis=0).tolist()
da_cou[0] = 'Doña Ana County, NM'

da_cou.insert(2, 10)

cou_crm = cou_crm.append(pd.DataFrame([da_cou], columns=cou_head), ignore_index=True, sort=False)
# shannon county, sd --> oglala lakota county, sd (name changed in 2015)
cou_crm = cou_crm.replace({'county_name': {'Shannon County, SD': 'Oglala Lakota County, SD'}})

In [10]:
cou_crm = pd.merge(couid,
                 cou_crm,
                 left_on='COU-ST',
                 right_on='county_name')

In [11]:
cou_names = {}

for cou in crm.iloc[:,0]:
    cou_names[cou] = cou[:-4]

cou_crm = cou_crm.replace({'county_name': cou_names})
cou_crm = cou_crm.rename({'crime_rate_per_100000': 'CrmRatePer100k',
                          'county_name': 'County',
                          'population': 'Pop'}, 
                         axis='columns')

In [12]:
crm_cols = ['CountyId', 'COU-ST', 'County', 'CrmRatePer100k',
            'MURDER', 'RAPE', 'ROBBERY', 'AGASSLT', 'BURGLRY', 'LARCENY', 'Pop']

cou_crm = cou_crm[crm_cols]

In [13]:
v_crm = cou_crm[['MURDER', 'RAPE', 'ROBBERY', 'AGASSLT', 'BURGLRY', 'LARCENY']]
v_crm = v_crm.sum(axis=1).tolist()

cou_crm['SumVCrm'] = v_crm

In [14]:
cou_crm['SumVCrm/Pop'] = cou_crm['SumVCrm']/cou_crm['Pop']
cou_crm['%SumVCrm/Pop'] = cou_crm['SumVCrm/Pop']*100

In [15]:
cou_crm

,CountyId,COU-ST,County,CrmRatePer100k,MURDER,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,Pop,SumVCrm,SumVCrm/Pop,%SumVCrm/Pop
0,1001,"Autauga County, AL",Autauga County,251.601926,3.0,15.0,34.0,87.0,447.0,1233.0,55246.0,1819.0,0.032925,3.292546
1,1003,"Baldwin County, AL",Baldwin County,228.086325,8.0,30.0,76.0,332.0,967.0,3829.0,195540.0,5242.0,0.026808,2.680781
2,1005,"Barbour County, AL",Barbour County,177.278771,0.0,4.0,8.0,36.0,90.0,362.0,27076.0,500.0,0.018467,1.846654
3,1007,"Bibb County, AL",Bibb County,217.661692,1.0,4.0,8.0,36.0,122.0,251.0,22512.0,422.0,0.018746,1.874556
4,1009,"Blount County, AL",Blount County,210.810064,1.0,11.0,9.0,101.0,397.0,865.0,57872.0,1384.0,0.023915,2.391485
5,1011,"Bullock County, AL",Bullock County,526.365260,0.0,2.0,12.0,43.0,98.0,55.0,10639.0,210.0,0.019739,1.973870
6,1013,"Butler County, AL",Butler County,537.873180,6.0,5.0,11.0,87.0,182.0,408.0,20265.0,699.0,0.034493,3.449297
7,1015,"Calhoun County, AL",Calhoun County,580.797697,6.0,37.0,131.0,504.0,1710.0,3482.0,116736.0,5870.0,0.050284,5.028440
8,1017,"Chambers County, AL",Chambers County,562.027984,1.0,7.0,24.0,160.0,374.0,875.0,34162.0,1441.0,0.042181,4.218137
9,1019,"Cherokee County, AL",Cherokee County,415.982903,1.0,4.0,2.0,102.0,329.0,610.0,26203.0,1048.0,0.039995,3.999542


In [16]:
result = pd.merge(cou_crm,
                  coucsv.drop(['County'], axis=1),
                  on='CountyId')
result = result.rename({'CountyId': 'CouId'}, axis='columns')
result = result.drop(['Unnamed: 0', 'COU-ST'], axis=1)

In [22]:
result.to_csv('cou_crm.csv')

In [17]:
result

,CouId,County,CrmRatePer100k,MURDER,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,Pop,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Autauga County,251.601926,3.0,15.0,34.0,87.0,447.0,1233.0,55246.0,...,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,1003,Baldwin County,228.086325,8.0,30.0,76.0,332.0,967.0,3829.0,195540.0,...,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
2,1005,Barbour County,177.278771,0.0,4.0,8.0,36.0,90.0,362.0,27076.0,...,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
3,1007,Bibb County,217.661692,1.0,4.0,8.0,36.0,122.0,251.0,22512.0,...,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
4,1009,Blount County,210.810064,1.0,11.0,9.0,101.0,397.0,865.0,57872.0,...,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9
5,1011,Bullock County,526.365260,0.0,2.0,12.0,43.0,98.0,55.0,10639.0,...,6.2,1.7,3.0,29.8,4290,81.4,13.6,5.0,0.0,12.1
6,1013,Butler County,537.873180,6.0,5.0,11.0,87.0,182.0,408.0,20265.0,...,0.9,0.9,2.0,23.2,7727,79.1,15.3,5.3,0.3,7.6
7,1015,Calhoun County,580.797697,6.0,37.0,131.0,504.0,1710.0,3482.0,116736.0,...,1.3,1.1,3.2,24.8,47392,74.9,19.9,5.1,0.1,10.1
8,1017,Chambers County,562.027984,1.0,7.0,24.0,160.0,374.0,875.0,34162.0,...,0.6,0.5,2.0,23.6,14527,84.5,11.8,3.7,0.0,6.4
9,1019,Cherokee County,415.982903,1.0,4.0,2.0,102.0,329.0,610.0,26203.0,...,0.3,0.3,2.0,26.5,9879,74.8,17.1,8.1,0.0,5.3


In [18]:
# TESTS BELOW

In [19]:
# test to find the missing county
# missing county = Doña Ana County

In [20]:
# c_test = crm[crm['county_name'].str.contains('County')]['county_name']
# c1_test = c_test.sort_values(ascending=True).reset_index()['county_name'].tolist()
# c1_test

# test01 = pd.read_csv('counties.csv')
# test02 = test01['County'].sort_values(ascending=True).reset_index()['County'].tolist()

# for t in np.arange(len(c1_test)):
#     if c1_test[t] != test02[t]:
#         print(test02[t])
#         print(c1_test[t])
#         break

# pd_test.to_csv('ree')

# test01[test01['County'].str.contains('Ana')]